In [14]:
# Install dependencies
!pip install -q sacrebleu unbabel-comet transformers[torch] datasets evaluate sentencepiece

# Standard Library Imports
import os
import math
import random
import shutil
from collections import Counter

# Data Processing & Math
import numpy as np
import pandas as pd
import torch

# Hugging Face Libraries
import evaluate
from datasets import Dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

# Comet Library
from comet import download_model, load_from_checkpoint

print("Environment setup and imports completed.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 21.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 4.25.8 which is incompatible.
ray 2.52.1 requires click!=8.3.*,>=7.0, but you have click 8.3.1 wh

In [2]:
# import os
# import math
# import random
# from collections import Counter

POSSIBLE_PATHS = [
    "/kaggle/input/final-data/processed", 
    "/kaggle/input/final-data", 
    "/kaggle/input/processed"
]
DATA_DIR = None
for p in POSSIBLE_PATHS:
    if os.path.exists(p) and "train_unpc.en" in os.listdir(p):
        DATA_DIR = p
        break

if not DATA_DIR:
    print("Data path not found. Please manually modify the DATA_DIR variable!")
else:
    print(f"Data Directory: {DATA_DIR}")

def analyze_corpus(name, src_filename, tgt_filename):
    src_path = os.path.join(DATA_DIR, src_filename)
    tgt_path = os.path.join(DATA_DIR, tgt_filename)
    
    if not os.path.exists(src_path) or not os.path.exists(tgt_path):
        print(f"⚠️ Skipping {name}: File not found")
        return

    print(f"\n======== Analyzing: {name} ========")
    
    # Statistical variables
    src_lines, tgt_lines = [], []
    src_lens_char, tgt_lens_char = [], []
    src_lens_word, tgt_lens_word = [], []
    src_vocab, tgt_vocab = set(), set()
    
    # Read Source
    with open(src_path, 'r', encoding='utf-8') as f:
        for line in f:
            l = line.strip()
            src_lines.append(l)
            src_lens_char.append(len(l))
            words = l.split() # Simple whitespace tokenization
            src_lens_word.append(len(words))
            src_vocab.update(words)

    # Read Target
    with open(tgt_path, 'r', encoding='utf-8') as f:
        for line in f:
            l = line.strip()
            tgt_lines.append(l)
            tgt_lens_char.append(len(l))
            words = l.split()
            tgt_lens_word.append(len(words))
            tgt_vocab.update(words)

    count = len(src_lines)
    print(f"1. Line Count (Sentence Pairs): {count}")
    
    avg_src_w = sum(src_lens_word) / count
    avg_tgt_w = sum(tgt_lens_word) / count
    avg_src_c = sum(src_lens_char) / count
    avg_tgt_c = sum(tgt_lens_char) / count
    
    print(f"2. Avg Length (Source EN): {avg_src_w:.2f} words, {avg_src_c:.2f} chars")
    print(f"   Avg Length (Target ZH): {avg_tgt_w:.2f} words, {avg_tgt_c:.2f} chars")

    print(f"3. Vocabulary Size (Unique Tokens): Source={len(src_vocab)}, Target={len(tgt_vocab)}")

    ratios = []
    for sl, tl in zip(src_lens_char, tgt_lens_char):
        if tl == 0: continue
        ratios.append(sl / tl)
    if ratios:
        avg_ratio = sum(ratios) / len(ratios)
        print(f"4. Avg Length Ratio (Src/Tgt Char Ratio): {avg_ratio:.2f}")

    def get_entropy(text_list):
        full_text = "".join(text_list)
        if not full_text: return 0
        counts = Counter(full_text)
        total = len(full_text)
        ent = 0
        for cnt in counts.values():
            p = cnt / total
            ent -= p * math.log2(p)
        return ent

    print("5. Calculating Character Entropy...")
    print(f"   Source Entropy: {get_entropy(src_lines):.4f}")
    print(f"   Target Entropy: {get_entropy(tgt_lines):.4f}")

    print("\n----- Random Sample (Qualitative Check) -----")
    indices = random.sample(range(count), min(20, count))
    for idx in indices:
        print(f"[{idx}] EN: {src_lines[idx]}")
        print(f"      ZH: {tgt_lines[idx]}")

if DATA_DIR:
    # Analyze UNPC Cleaned
    analyze_corpus("UNPC Cleaned (Train)", "train_unpc.en", "train_unpc.zh")
    
    # Analyze TED Cleaned
    analyze_corpus("TED Cleaned (Train)", "train_ted.en", "train_ted.zh")

Data Directory: /kaggle/input/final-data/processed

======== Analyzing: UNPC Cleaned (Train) ========
1. Line Count (Sentence Pairs): 462483
2. Avg Length (Source EN): 19.45 words, 123.11 chars
   Avg Length (Target ZH): 2.81 words, 49.63 chars
3. Vocabulary Size (Unique Tokens): Source=372300, Target=669711
4. Avg Length Ratio (Src/Tgt Char Ratio): 2.42
5. Calculating Character Entropy...
   Source Entropy: 4.8487
   Target Entropy: 8.5990

----- Random Sample (Qualitative Check) -----
[235377] EN: (d) Human rights and disability;
      ZH: (d) 人权与残疾问题；
[401839] EN: The Council then proceeded to the vote on the draft resolution before it (S/2000/1124).
      ZH: 安理会接着就面前的决议草案（S/2000/1124）进行表决。
[422352] EN: B. Organization of UNISPACE III
      ZH: B. 第三次外空会议的安排
[293831] EN: In May 1999, UNFPA approved its first project for the former Yugoslav Republic of Macedonia, including an initial amount of $5,000 earmarked for technical needs assessment.
      ZH: 1999年5月,人口基金核准了其援助前南斯拉夫的马其顿共和国的

In [3]:
#!pip install -q transformers[torch] datasets sacrebleu evaluate sentencepiece

#import torch

print(f"GPU Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Current Device: {torch.cuda.get_device_name(0)}")
else:
    print("Warning: You are using CPU!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
GPU Available: True
Current Device: Tesla T4


In [4]:
# Tokenizer Analysis & Data Path Verification
#import os
#import numpy as np
#from transformers import AutoTokenizer

POSSIBLE_PATHS = [
    "/kaggle/input/final-data/processed", 
    "/kaggle/input/final-data",          
    "/kaggle/input/processed"            
]

DATA_DIR = None
for p in POSSIBLE_PATHS:
    if os.path.exists(p) and "train_unpc.en" in os.listdir(p):
        DATA_DIR = p
        break

if not DATA_DIR:
    print("Data not found! Please check the Data panel on the right, copy the 'processed' folder path, and update the paths above.")
else:
    print(f"Data path confirmed: {DATA_DIR}")

MODEL_CHECKPOINT = "Helsinki-NLP/opus-mt-en-mul"

# Load Tokenizer
print(f"Loading model vocabulary: {MODEL_CHECKPOINT}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def analyze_file(filename, label):
    filepath = os.path.join(DATA_DIR, filename)
    print(f"\n Analyzing: {label} ({filename})")
    
    lengths = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 10000: break 
            tokens = tokenizer.tokenize(line.strip())
            lengths.append(len(tokens))
            
            if i < 2:
                print(f"  [Sample {i+1}] Text: {line.strip()[:50]}...")
                print(f"  [Sample {i+1}] Tokens: {tokens}")

    print(f" {label} Average Length: {np.mean(lengths):.2f} tokens")

# Run Analysis
if DATA_DIR:
    analyze_file("train_unpc.en", "English Train Set")
    analyze_file("train_unpc.zh", "Chinese Train Set")

Data path confirmed: /kaggle/input/final-data/processed
Loading model vocabulary: Helsinki-NLP/opus-mt-en-mul...


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")



 Analyzing: English Train Set (train_unpc.en)
  [Sample 1] Text: a 1999 data are provisional....
  [Sample 1] Tokens: ['▁a', '▁1999', '▁data', '▁are', '▁provisional', '.']
  [Sample 2] Text: Recalling its resolution 49/251 of 20 July 1995 on...
  [Sample 2] Tokens: ['▁Recalling', '▁its', '▁resolution', '▁49', '/25', '1', '▁of', '▁20', '▁July', '▁1995', '▁on', '▁the', '▁financing', '▁of', '▁the', '▁Tribunal', '▁and', '▁its', '▁subsequent', '▁resolutions', '▁thereon', ',', '▁the', '▁latest', '▁of', '▁which', '▁was', '▁resolution', '▁53', '/21', '3', '▁of', '▁18', '▁December', '▁1998,']
 English Train Set Average Length: 27.84 tokens

 Analyzing: Chinese Train Set (train_unpc.zh)
  [Sample 1] Text: a 1999年数据为暂定数据。...
  [Sample 1] Tokens: ['▁a', '▁1999', '年', '数', '据', '为', '暂', '定', '数', '据', '。']
  [Sample 2] Text: 回顾其1995年7月20日关于该法庭经费筹措的第49/251号决议及其后各项有关决议,最近的一项是1...
  [Sample 2] Tokens: ['▁', '回', '顾', '其', '1995', '年', '7', '月', '20', '日', '关', '于', '该', '法', '庭', '经', '费', '筹措', '的'

In [12]:
DATA_DIR = "/kaggle/input/final-data/processed" 

CHECKPOINT_PATH = "/kaggle/input/final-unpc-model" 

OUTPUT_DIR = "/kaggle/working/final_ted_model"

def load_dataset_from_text(src_path, tgt_path):
    with open(src_path, "r", encoding="utf-8") as fs, open(tgt_path, "r", encoding="utf-8") as ft:
        return Dataset.from_dict({"source": [l.strip() for l in fs], "target": [l.strip() for l in ft]})

SOURCE_PREFIX = ">>zho<< "

def preprocess_function(examples):
    inputs = [SOURCE_PREFIX + ex for ex in examples["source"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple): preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

print(f"Loading Pre-trained UNPC Model from: {CHECKPOINT_PATH} ...")

try:
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_PATH)
    model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT_PATH)
    print("Successfully loaded UNPC model weights!")
except OSError as e:
    print("Error loading model. Please check the path in Input panel.")
    raise e

# Load TED data
print("Loading TED data...")
train_ted = load_dataset_from_text(
    os.path.join(DATA_DIR, "train_ted.en"), 
    os.path.join(DATA_DIR, "train_ted.zh")
)
dev_ted = load_dataset_from_text(
    os.path.join(DATA_DIR, "dev_ted.en"), 
    os.path.join(DATA_DIR, "dev_ted.zh")
)
print(f"TED Train size: {len(train_ted)}")

print("Preprocessing TED data...")
tokenized_train_ted = train_ted.map(preprocess_function, batched=True)
tokenized_dev_ted = dev_ted.map(preprocess_function, batched=True)

# Set training arguments (TED Domain Adaptation)
args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",      
    save_strategy="epoch",
    learning_rate=2e-5,        
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,         
    predict_with_generate=True,
    fp16=True,
    logging_steps=50,        
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_ted,
    eval_dataset=tokenized_dev_ted,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Starting TED Domain Adaptation...")
trainer.train()

trainer.save_model(OUTPUT_DIR)
print(f"TED Training Complete! Model saved to {OUTPUT_DIR}")


shutil.make_archive("/kaggle/working/ted_model_backup", 'zip', OUTPUT_DIR)
print("Backup created: ted_model_backup.zip")

Loading Pre-trained UNPC Model from: /kaggle/input/final-unpc-model ...
Successfully loaded UNPC model weights!
Loading TED data...
TED Train size: 50000
Preprocessing TED data...


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/tmp/ipykernel_55/1163181332.py:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Starting TED Domain Adaptation...


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,2.547800,2.380073,18.333715
2,2.368200,2.317173,18.197097
3,2.279800,2.298169,17.701789


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TED Training Complete! Model saved to /kaggle/working/final_ted_model
Backup created: ted_model_backup.zip


In [16]:
#import os
#import torch
#import numpy as np
#import pandas as pd
#from datasets import Dataset
#from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
#import evaluate
#from comet import download_model, load_from_checkpoint


DATA_DIR = "/kaggle/input/final-data/processed"
TATOEBA_FILE_PATH = "/kaggle/input/tatoeba-test/tatoeba-test-v2023-09-26.eng-zho.txt"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODELS = {
    "Baseline": "Helsinki-NLP/opus-mt-en-mul",
    "Model A (UNPC)": "/kaggle/input/final-unpc-model", 
    "Model B (TED)":  "/kaggle/input/trained-ted-model"   
}

TEST_SETS = {
    "TED Test": ("pair", ("test_ted.en", "test_ted.zh")),
    "Tatoeba":  ("single", TATOEBA_FILE_PATH) 
}

# Initialize Metrics
# BLEU & chrF 
metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")

# COMET 
print("Downloading COMET model...")
try:
    comet_model_path = download_model("Unbabel/wmt20-comet-da")
    comet_model = load_from_checkpoint(comet_model_path)
    use_comet = True
except Exception as e:
    print(f"COMET load failed: {e}. Skipping COMET.")
    use_comet = False


# Load standard pair files (TED/UNPC)
def load_pair_data(src_file, tgt_file):
    with open(os.path.join(DATA_DIR, src_file), 'r') as f: src = [l.strip() for l in f]
    with open(os.path.join(DATA_DIR, tgt_file), 'r') as f: tgt = [l.strip() for l in f]
    return src, tgt

# Load Tatoeba single file
def load_tatoeba_file(file_path):
    print(f"Reading Tatoeba from: {file_path}")
    srcs = []
    tgts = []
    
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return [], []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            
            if len(parts) >= 4 and 'Hans' in parts[1]:
                srcs.append(parts[2])
                tgts.append(parts[3])
                
    print(f"Loaded {len(srcs)} sentences from Tatoeba (Simplified Chinese only).")
    return srcs, tgts

# Core Evaluation Function
def evaluate_model(model_path, model_name, test_name, src_text, tgt_text):
    print(f"Evaluating {model_name} on {test_name}...")
    
    # Load model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(DEVICE)
    
    # Generate Translations
    translations = []
    batch_size = 16 
    prefix = ">>zho<< "
    
    for i in range(0, len(src_text), batch_size):
        batch = [prefix + s for s in src_text[i:i+batch_size]]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(DEVICE)
        
        with torch.no_grad():
            generated = model.generate(**inputs, max_length=128)
        
        decoded = tokenizer.batch_decode(generated, skip_special_tokens=True)
        translations.extend(decoded)
    
    # Calculate BLEU (tokenized)
    bleu_score = metric_bleu.compute(predictions=translations, references=tgt_text, tokenize='zh')['score']
    
    # Calculate chrF (character level)
    chrf_score = metric_chrf.compute(predictions=translations, references=tgt_text)['score']
    
    # Calculate COMET
    comet_score = 0
    if use_comet:
        comet_data = [{"src": s, "mt": m, "ref": r} for s, m, r in zip(src_text, translations, tgt_text)]

        try:
            comet_out = comet_model.predict(comet_data, batch_size=8, gpus=1)
            comet_score = comet_out.system_score 
        except Exception as e:
            print(f"COMET calculation error: {e}")
    
    # Print Examples
    print(f"Examples for {model_name}:")
    for i in range(min(3, len(src_text))): 
        print(f"   Src: {src_text[i]}")
        print(f"   Ref: {tgt_text[i]}")
        print(f"   Hyp: {translations[i]}")
        print("-" * 20)

    return {
        "BLEU": round(bleu_score, 2),
        "chrF": round(chrf_score, 2),
        "COMET": round(comet_score, 4)
    }

# Main Loop
final_results = []

for model_name, model_path in MODELS.items():
    for test_name, (load_type, file_info) in TEST_SETS.items():
        
        if load_type == "pair":
            src, tgt = load_pair_data(file_info[0], file_info[1])
        elif load_type == "single":
            src, tgt = load_tatoeba_file(file_info)
        
        if len(src) > 0:
            scores = evaluate_model(model_path, model_name, test_name, src, tgt)
            
            res = {"Model": model_name, "Test Set": test_name}
            res.update(scores)
            final_results.append(res)
            
            torch.cuda.empty_cache()

df = pd.DataFrame(final_results)
print("FINAL RESULTS TABLE:")
print(df.to_markdown(index=False))

df.to_csv("evaluation_results.csv", index=False)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../root/.cache/huggingface/hub/models--Unbabel--wmt20-comet-da/snapshots/87819f4d6d4f17e0d1752cc9e0ccfa2064997219/checkpoints/model.ckpt`
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/saving.py:197: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


Evaluating Baseline on TED Test...


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

...

Examples for Model A (UNPC):
   Src: Tom got drunk with his friends in the park.
   Ref: 汤姆和他的朋友们在公园喝醉了
   Hyp: Tom与他在公园的朋友喝醉了。
--------------------
   Src: Tom isn't going to give me that.
   Ref: 汤姆不会同意的
   Hyp: 托姆不会给我这样。
--------------------
   Src: 2016 was the hottest year on record.
   Ref: 2016年是有记录以来最热的一年。
   Hyp: 2016年是记录最热的年份。
--------------------
Evaluating Model B (TED) on TED Test...


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Predicting DataLoader 0: 100%|██████████| 125/125 [00:23<00:00,  5.29it/s]


Examples for Model B (TED):
   Src: So Europe is not just an example now to emulate; it's an enemy to fight and to resist.
   Ref: 因此欧洲从之前的榜样变成了当时的竞争者 变成了需要反抗和斗争的敌人
   Hyp: 所以欧洲不仅仅是一个模拟的例子, 而是一种战斗和抵抗的敌人。
--------------------
   Src: I do want to talk -- the absolute icing on this cemetery cake is the Barricini family mausoleum nearby.
   Ref: 但我想说 -- 公墓所用蛋糕的糖衣部分 是在Barricini 家族的陵墓附近.
   Hyp: 我想说话—— 这个墓地蛋糕上的绝对兴奋 是巴里西尼的家属毛索里姆附近。
--------------------
   Src: It's a parabolic Scheffler solar cooker.
   Ref: 这是一个抛物形的全自动太阳能炊具。
   Hyp: 它是一名模拟性的Scheffler太阳能厨师。
--------------------
Reading Tatoeba from: /kaggle/input/tatoeba-test/tatoeba-test-v2023-09-26.eng-zho.txt
Loaded 5526 sentences from Tatoeba (Simplified Chinese only).
Evaluating Model B (TED) on Tatoeba...


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Predicting DataLoader 0: 100%|██████████| 691/691 [01:16<00:00,  9.05it/s]


Examples for Model B (TED):
   Src: Tom got drunk with his friends in the park.
   Ref: 汤姆和他的朋友们在公园喝醉了
   Hyp: 汤姆和他在公园里的朋友喝醉了。
--------------------
   Src: Tom isn't going to give me that.
   Ref: 汤姆不会同意的
   Hyp: (笑声) 汤姆不会给我这件事。
--------------------
   Src: 2016 was the hottest year on record.
   Ref: 2016年是有记录以来最热的一年。
   Hyp: 2016年是记录上最热的一年。
--------------------
FINAL RESULTS TABLE:
| Model          | Test Set   |   BLEU |   chrF |   COMET |
|:---------------|:-----------|-------:|-------:|--------:|
| Baseline       | TED Test   |  14.19 |  17.83 | -0.3222 |
| Baseline       | Tatoeba    |  20.87 |  18.57 |  0.2637 |
| Model A (UNPC) | TED Test   |  16.07 |  18.92 | -0.1693 |
| Model A (UNPC) | Tatoeba    |  19.48 |  18.99 |  0.3034 |
| Model B (TED)  | TED Test   |  21.01 |  24.72 |  0.0018 |
| Model B (TED)  | Tatoeba    |  21.07 |  21.83 |  0.2977 |
